# Cryptocompare API Exercise

Going through the API notebook using the [Cryptocompare.com](https://www.cryptocompare.com/) API as it is relevant to the question I want to answer in my EDA project.

In [1]:
import requests

In [2]:
# Make a request to get historical Bitcoin data
response = requests.get('https://min-api.cryptocompare.com/data/histoday')

print(response.status_code)

200


In [3]:
params = {'fsym': 'BTC', 'tsym': 'USD', 'limit': 100}

response = requests.get('https://min-api.cryptocompare.com/data/histoday', params=params)

print(response.content)

b'{"Response":"Success","Type":100,"Aggregated":false,"Data":[{"time":1512518400,"close":13749.57,"high":13843.2,"low":11661.76,"open":11667.13,"volumefrom":191576.66,"volumeto":2437037791.81},{"time":1512604800,"close":16850.31,"high":16879.26,"low":13401.61,"open":13750.09,"volumefrom":297108.66,"volumeto":4510225316.19},{"time":1512691200,"close":16047.61,"high":17294.85,"low":13906.1,"open":16867.98,"volumefrom":286762.02,"volumeto":4546014731.58},{"time":1512777600,"close":14843.42,"high":16313.18,"low":13151.47,"open":16048.18,"volumefrom":181979.81,"volumeto":2699876215.33},{"time":1512864000,"close":15059.6,"high":15783.2,"low":13031,"open":14839.98,"volumefrom":201620.09,"volumeto":2904037859.64},{"time":1512950400,"close":16732.47,"high":17399.18,"low":15024.56,"open":15060.45,"volumefrom":159724.56,"volumeto":2634267594.52},{"time":1513036800,"close":17083.9,"high":17560.65,"low":16254.53,"open":16733.29,"volumefrom":132846.57,"volumeto":2246138895.89},{"time":1513123200,"cl

In [4]:
import json

In [5]:
data = response.json()['Data']
print(data[0])

{'time': 1512518400, 'close': 13749.57, 'high': 13843.2, 'low': 11661.76, 'open': 11667.13, 'volumefrom': 191576.66, 'volumeto': 2437037791.81}


In [6]:
import pandas as pd
import datetime

In [9]:
df = pd.DataFrame(data)
df['date'] = [datetime.date.fromtimestamp(d) for d in df.time]

In [10]:
df.head()

,close,high,low,open,time,volumefrom,volumeto,date
0,13749.57,13843.20,11661.76,11667.13,1512518400,191576.66,2.437038e+09,2017-12-05
1,16850.31,16879.26,13401.61,13750.09,1512604800,297108.66,4.510225e+09,2017-12-06
2,16047.61,17294.85,13906.10,16867.98,1512691200,286762.02,4.546015e+09,2017-12-07
3,14843.42,16313.18,13151.47,16048.18,1512777600,181979.81,2.699876e+09,2017-12-08
4,15059.60,15783.20,13031.00,14839.98,1512864000,201620.09,2.904038e+09,2017-12-09


In [11]:
df = df[['open', 'high', 'low', 'close', 'volumeto', 'date']]
df.head()

,open,high,low,close,volumeto,date
0,11667.13,13843.20,11661.76,13749.57,2.437038e+09,2017-12-05
1,13750.09,16879.26,13401.61,16850.31,4.510225e+09,2017-12-06
2,16867.98,17294.85,13906.10,16047.61,4.546015e+09,2017-12-07
3,16048.18,16313.18,13151.47,14843.42,2.699876e+09,2017-12-08
4,14839.98,15783.20,13031.00,15059.60,2.904038e+09,2017-12-09


In [27]:
# Create list of coins that we want databases for
coin_list = ['BTC', 'ETH', 'XRP', 'BCH', 'LTC', 'ADA', 'NEO', 'XLM', 'EOS', 'XMR']

# Create a function to generate desired dataframe for given coin name
def generate_historical_dataframe(sym, curr='USD', limit=100):
    url = 'https://min-api.cryptocompare.com/data/histoday'
    params = {'fsym': sym, 'tsym': curr, 'limit': limit}
    
    response = requests.get(url, params=params)
    data = response.json()['Data']
    df = pd.DataFrame(data)
    df['date'] = [datetime.date.fromtimestamp(d) for d in df.time]
    df = df[['open', 'high', 'low', 'close', 'volumeto', 'date']]
    
    return df

In [28]:
import sqlite3

# Write function to create a database for all coins in coinlist
def create_database(df, table_name, sqlite_file):
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()

    df.to_sql(table_name, conn, if_exists='replace')
    
    conn.commit()
    conn.close()

In [31]:
create_database(df, "TEST1", 'test_db.sqlite')

In [32]:
# Name of the database file
sqlite_file = 'cryptocompare_db.sqlite'

# Loop through coin list, get historical data, create new database
for coin in coin_list:
    coin_df = generate_historical_dataframe(coin)
    create_database(coin_df, coin, sqlite_file)

# Create a Securities Master Database 

## Schema 

### 1. Data - id | name | url 

### 2. Symbol - id | data_id | ticker | name | sector | exchange

### 3. Daily_Price -

#### id | data_id | symbol_id | date | open | high | low | close | volume | atr | N | range_high | range_low | ma5 | ma10 | ma20 | ma50 | ma100 | ma200

In [16]:
import sqlite3

# Initialize variables for file name, table, columns, data types 
sqlite_file = 'securities_master_db.sqlite'
table_name = 'Data'
id_col = 'id'
name_col = 'name'
url_col = 'url'
dtype_int = 'INTEGER'
dtype_text = 'TEXT'

# Connect to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# Create a new table with 3 columns
c.execute('CREATE TABLE {tn} ({ic} {dti} PRIMARY KEY, {nc} {dtt}, {uc} {dtt})'\
         .format(tn=table_name, ic=id_col, dti=dtype_int, nc=name_col, dtt=dtype_text, uc=url_col))

# Commit changes and close
conn.commit()
conn.close()

In [14]:
# Drop table script to test
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

c.execute('DROP TABLE {tn}'.format(tn=table_name))

conn.commit()
conn.close()

In [17]:
# Define connect and close functions so you dont have to keep repeating them
def connect(sqlite_file):
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    return conn, c

def close(conn):
    conn.commit()
    conn.close()

In [18]:
# Add value for Cryptocompare to Data table
conn, c = connect(sqlite_file)

c.execute("INSERT INTO {tn} ({ic}, {nc}, {uc}) VALUES (1, 'Cryptocompare', 'https://min-api.cryptocompare.com')"\
          .format(tn=table_name, ic=id_col, nc=name_col, uc=url_col))

close(conn)

In [19]:
# Add value for Quandl to Data table
conn, c = connect(sqlite_file)

c.execute("INSERT INTO {tn} ({ic}, {nc}, {uc}) VALUES (2, 'Quandl', 'https://docs.quandl.com')"\
          .format(tn=table_name, ic=id_col, nc=name_col, uc=url_col))

close(conn)

In [24]:
sqlite_file = 'securities_master_db.sqlite'
table_name = 'Data'
id_col = 'id'
name_col = 'name'
url_col = 'url'
dtype_int = 'INTEGER'
dtype_text = 'TEXT'
# Add value for Quantopian to Data table
conn, c = connect(sqlite_file)

c.execute("INSERT INTO {tn} ({ic}, {nc}, {uc}) VALUES (3, 'Quantopian', 'https://www.quantopian.com/data')"\
          .format(tn=table_name, ic=id_col, nc=name_col, uc=url_col))

close(conn)

In [21]:
# Create table for symbol
# Initialize variables for file name, table, columns, data types 
sqlite_file = 'securities_master_db.sqlite'

table_name = 'Symbol'
id_col = 'id'
data_id_col = 'data_id'
ticker_col = 'ticker'
name_col = 'name'
sector_col = 'sector'
exchange_col = 'exchange'
dtype_int = 'INTEGER'
dtype_text = 'TEXT'

# Connect to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# Create a new table with 3 columns
c.execute('CREATE TABLE {tn} ({ic} {dti} PRIMARY KEY,\
                              {dc} {dti},\
                              {tc} {dtt},\
                              {nc} {dtt},\
                              {sc} {dtt},\
                              {ec} {dtt})'\
         .format(tn=table_name, ic=id_col, dti=dtype_int, dc=data_id_col, tc=ticker_col, dtt=dtype_text,\
                 nc=name_col, sc=sector_col, ec=exchange_col))

# Commit changes and close
conn.commit()
conn.close()

In [22]:
# Create table for daily_price
# Initialize variables for file name, table, columns, data types 
sqlite_file = 'securities_master_db.sqlite'

table_name = 'Daily_Price'

id_col = 'id'
data_id_col = 'data_id'
symbol_id_col = 'symbol_id'
date_col = 'date'
open_col = 'open'
high_col = 'high'
low_col = 'low'
close_col = 'close'
volume_col = 'volume'
atr_col = 'atr'
N_col = 'N'
range_high_col = 'range_high'
range_low_col = 'range_low'
ma5_col = 'ma5'
ma10_col = 'ma10'
ma20_col = 'ma20'
ma50_col = 'ma50'
ma100_col = 'ma100'
ma200_col = 'ma200'

dtype_int = 'INTEGER'
dtype_text = 'TEXT'
dtype_real = 'REAL'

# Connect to the database file
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

# Create a new table with 3 columns
c.execute('CREATE TABLE {tn} ({ic} {dti} PRIMARY KEY,\
                              {dc} {dti},\
                              {sc} {dti},\
                              {dtc} {dtt},\
                              {oc} {dtr},\
                              {hc} {dtr},\
                              {lc} {dtr},\
                              {cc} {dtr},\
                              {vc} {dti},\
                              {ac} {dtr},\
                              {Nc} {dtr},\
                              {rhc} {dtr},\
                              {rlc} {dtr},\
                              {m5c} {dtr},\
                              {m10c} {dtr},\
                              {m20c} {dtr},\
                              {m50c} {dtr},\
                              {m100c} {dtr},\
                              {m200c} {dtr})'\
         .format(tn=table_name, ic=id_col, dti=dtype_int, dc=data_id_col, sc=symbol_id_col, dtc=date_col,\
                 dtt=dtype_text, oc=open_col, dtr=dtype_real, hc=high_col, lc=low_col, cc=close_col, vc=volume_col,\
                 ac=atr_col, Nc=N_col, rhc=range_high_col, rlc=range_low_col, m5c=ma5_col, m10c=ma10_col,\
                 m20c=ma20_col, m50c=ma50_col, m100c=ma100_col, m200c=ma200_col))

# Commit changes and close
conn.commit()
conn.close()